In [1]:
# Set up Notebook
% matplotlib inline

# Standard imports
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import cm


# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
labels = ['clust_logFCS', 'clust_RCSI', 'clust_HDDS']

In [3]:
x_train=train_data.drop(labels, axis=1)
x_train = x_train.drop(x_train.columns[0], axis=1)
x_test=test_data.drop(labels, axis=1)
x_test = x_test.drop(x_test.columns[0], axis=1)

x_train.head()

,clust_L12raincytot,clust_L12day1rain,clust_L12maxdays,clust_floodmax,clust_cells_own,clust_price,clust_thinn,clust_roof,clust_hhsize,clust_hh_age,clust_hh_gender,clust_asset,clust_dist_road,clust_dist_admarc,clust_percent_ag,clust_nutri_reten_constrained,clust_elevation,ipc_lag1,ipc_lag12
0,1090.06130,43,17,0.0,0.1250,3.824215,0.359375,0.1250,5.4375,41.6250,1.2500,-0.327686,1.395625,5.176875,0.6000,0.0000,1311.8750,1.0,NaN
1,855.86176,58,40,0.0,0.4375,3.917409,0.250000,0.1875,5.6875,36.5625,1.3125,-0.202549,0.756875,5.408750,0.6000,0.9375,496.0000,1.0,NaN
2,1300.24370,53,29,0.0,1.5000,3.683867,0.250000,0.5000,6.3750,40.4375,1.3750,0.548275,0.163125,17.038126,0.4500,1.0000,526.7500,1.0,NaN
3,1036.97120,52,29,0.0,0.6250,4.061391,0.500000,0.2500,5.4375,46.8750,1.1875,-0.077412,5.803750,7.724375,0.4375,0.0000,564.1875,1.0,NaN
4,952.18280,54,25,0.0,0.5625,3.879500,0.250000,0.0625,7.0000,43.8125,1.0000,-0.202549,11.286875,11.103125,0.4875,1.0000,1539.9375,1.0,NaN


In [4]:
y_train = train_data[labels]
y_test = test_data[labels]
y_test.head()

,clust_logFCS,clust_RCSI,clust_HDDS
0,3.666015,12.312500,4.687500
1,3.718879,5.352941,5.235294
2,3.799830,5.080000,5.720000
3,3.844891,4.285714,5.095238
4,3.687936,0.812500,5.312500


In [5]:
from sklearn.preprocessing import Imputer,StandardScaler

imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [6]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
x_train = poly.fit_transform(x_train)
x_test =  poly.fit_transform(x_test)



In [7]:
from sklearn.linear_model import LinearRegression

# Create and fit our linear regression model to training data
model = LinearRegression(fit_intercept=True)
model.fit(x_train, y_train[labels[0]])

# Compute model predictions for test data
pred = model.predict(x_test)

actual = y_test[labels[0]]
r2_linear = stats.pearsonr(actual, pred)[0] ** 2
r2_linear

0.31423980864791085

In [10]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True)

parameters = {'max_depth':np.arange( 1,4, 1 ).tolist(), 'min_samples_leaf':np.arange( 1, 4, 1 ).tolist()}
clf = GridSearchCV(rfc, parameters,cv=6, n_jobs= 4, iid = True,  refit= True,pre_dispatch= '2*n_jobs')
clf.fit(x_train, y_train[labels[0]])

# Fit estimator and display score

# Regress on test data
pred = clf.predict(x_test)

actual = y_test[labels[0]]
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

0.5387090912164537

In [10]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True,max_depth=4, min_samples_leaf=5 )

rfc.fit(x_train, y_train[labels[0]])
# Fit estimator and display score

# Regress on test data
pred = rfc.predict(x_test)

actual = y_test[labels[0]]
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

0.5500143975932646

In [38]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=(100,200,300,350), fit_intercept=True, normalize=False, scoring=None, cv=5, gcv_mode='auto', store_cv_values=False) 

# Define different alpha values for different fits
# alpha = [0.0, 1E-6, 1E-4, 1E-2, 1.0]

ridge.fit(x_train, y_train[labels[0]])
pred = ridge.predict(x_test)

actual = y_test[labels[0]]
r2_ridge= stats.pearsonr(actual, pred)[0] ** 2
r2_ridge

0.4651771428911595

In [39]:
from sklearn.linear_model import BayesianRidge
bridge = BayesianRidge(compute_score=True)
bridge.fit(x_train, y_train[labels[0]])
pred = bridge.predict(x_test)

actual = y_test[labels[0]]
r2_bridge= stats.pearsonr(actual, pred)[0] ** 2
r2_bridge

0.46622893732327914

In [107]:
from sklearn.linear_model import LassoCV

ls = LassoCV(eps=0.001, n_alphas=100, alphas=(0.0425, 0.045, 0.05, 0.04), fit_intercept=True,precompute='auto',n_jobs=4, random_state=0, selection='cyclic')

ls = ls.fit(x_train, y_train[labels[0]])
pred = ls.predict(x_test)

actual = y_test[labels[0]]
r2_ls= stats.pearsonr(actual, pred)[0] ** 2
r2_ls

0.5769390528828111

In [106]:
from sklearn.linear_model import ElasticNetCV
en = ElasticNetCV(alphas=(0.1,0.5,0.3, 1.0, 2.0,3,2.5), copy_X=True, cv=10, eps=0.004, fit_intercept=True,
       l1_ratio=0.493, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0)

en.fit(x_train, y_train[labels[0]])

pred = en.predict(x_test)

actual = y_test[labels[0]]
r2_en= stats.pearsonr(actual, pred)[0] ** 2
r2_en


0.5763028652825017

In [105]:
from sklearn.ensemble import GradientBoostingRegressor

# Create Regressor with default properties
gbr = GradientBoostingRegressor(random_state=0,learning_rate=0.1, n_estimators=8,subsample=1, criterion='friedman_mse', min_samples_split=2)

gbr.fit(x_train, y_train[labels[0]])
pred = gbr.predict(x_test)

actual = y_test[labels[0]]
r2_gbr= stats.pearsonr(actual, pred)[0] ** 2
r2_gbr


0.5767855254765067

In [108]:
df = pd.DataFrame(actual)
df["pred"] = pred
df.to_csv('FCSsplot.csv')

In [109]:
df

,clust_logFCS,pred
0,3.666015,3.713040
1,3.718879,3.747065
2,3.799830,3.942414
3,3.844891,3.817325
4,3.687936,3.803353
5,3.784152,3.786078
6,3.837426,3.743894
7,3.895304,3.739620
8,3.804709,3.704640
9,3.617289,3.805083
